In [ ]:

import numpy as np 
import cv2
from skimage.morphology import skeletonize
import copy
import AnalysisFuncs as AF
from astropy.io import fits
from reproject import reproject_exact, reproject_adaptive, reproject_interp


def processFilamentCenters(coords_data):
    kernel_size = 3
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    dilated_image = cv2.morphologyEx(coords_data, cv2.MORPH_CLOSE, kernel)
    dilated_image = skeletonize(dilated_image.astype(bool))
    coords_data = dilated_image.astype(np.uint8)



    #Step 4: Remove junctions
    fil_centers = copy.deepcopy(coords_data)
    junctions = AF.getSkeletonIntersection(np.array(fil_centers*255))
    IntersectsRemoved = AF.removeJunctions(junctions, fil_centers, dot_size = 1) #check intersects removed
    IntersectsRemoved[IntersectsRemoved > 0] = 1
    IntersectsRemoved[IntersectsRemoved < 0] = 0
    fil_centers = IntersectsRemoved
    
    return fil_centers, coords_data        


In [ ]:
def reprojectWrapper(inData, inHeader, OutputHeader, OutputData):
    """
    Reproject data from one fits file to another. Take OrigData and OrigHeader and reproject into the frame of OutputHeader and OutputData.

    Parameters:
    - OrigData: Data to reproject
    - OrigHeader: Header of data to reproject
    - OutputHeader: Header of data to reproject into
    - OutputData: Data to reproject into, used only for shape
    """

    reprojected_data, _ = reproject_interp((inData, inHeader), OutputHeader, shape_out=(np.shape(OutputData)))
    reprojected_data = cropNanBorder(reprojected_data, np.shape(OutputData))
    reprojected_data = np.nan_to_num(reprojected_data, nan=0.0)

    return reprojected_data

def cropNanBorder(image, expected_shape):

    """
    Crop the nan border away such that the image fits the expected shape

    Parameters: 
    -image (float): The data to crop. 2D array. 
    -expected_shape (int): the expected shape to crop the data into. 2 element tuple. 

    Returns: 
    - returns the cropped image
    """

    # Create a mask of non-NaN values
    image = np.array(image, dtype=float)  # converts numbers; non-convertible become nan
    mask = ~np.isnan(image)

    # Find the rows and columns with at least one non-NaN value
    non_nan_rows = np.where(mask.any(axis=1))[0]
    non_nan_cols = np.where(mask.any(axis=0))[0]

    # Use the min and max of these indices to slice the image
    cropped_image = image[non_nan_rows.min():non_nan_rows.max() + 1, non_nan_cols.min():non_nan_cols.max() + 1]
    
    # Get the current shape of the cropped image
    current_shape = cropped_image.shape
    
    # Check if the cropped image needs to be resized
    if current_shape != expected_shape:
        # Pad or crop to reach the expected shape
        padded_image = np.full(expected_shape, np.nan)  # Initialize with NaNs
        
        # If cropped_image is larger than expected_shape, trim it
        trim_rows = min(current_shape[0], expected_shape[0])
        trim_cols = min(current_shape[1], expected_shape[1])
        
        # Center the cropped image within the expected shape
        start_row = (expected_shape[0] - trim_rows) // 2
        start_col = (expected_shape[1] - trim_cols) // 2
        
        # Place the trimmed or centered cropped_image in the padded_image
        padded_image[start_row:start_row + trim_rows, start_col:start_col + trim_cols] = \
            cropped_image[:trim_rows, :trim_cols]
        
        return padded_image

In [ ]:
from multiprocessing import process


print('Begenning synthetic map production')
fits_path = r'/Users/jakehoffmann/Documents/JHU_Research/FilPHANGS_Main/ngc0628_F770W/Composites/ngc0628_F770W_JWST_Emission_starsub_CDDss0016pc_Composites.fits'

with fits.open(fits_path, ignore_missing=True) as hdul:
    coords_data = np.array(hdul[0].data)  # Assuming the image data is in the primary HDU 
    header = hdul[0].header

_, coords_data = processFilamentCenters(coords_data) #no Blocking on 16pc

out_path = r"/Users/jakehoffmann/Documents/JHU_Research/FilPHANGS_Main/ngc0628_F770W/Composites/_filCentersDilated.fits"
hdu = fits.PrimaryHDU(coords_data, header= header)
hdu.writeto(out_path, overwrite=True)